<a href="https://colab.research.google.com/github/rajimustafa100/Symbolic-Congruence/blob/main/Interaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rapidfuzz
import nltk
nltk.download("vader_lexicon")


In [ ]:
from google.colab import files

# Upload metadata.json and reviews.json files
uploaded = files.upload()



In [ ]:
from google.colab import files

# Upload metadata.json and reviews.json files
uploaded = files.upload()



In [ ]:
import pandas as pd
# Load metadata and reviews
metadata = pd.read_json("metadata.json", lines=True)
reviews = pd.read_json("reviews.json", lines=True)


In [ ]:
# Filter US-based reviews
us_reviews = reviews[reviews['reviewerLocation'].str.contains("USA|United States|US", na=False)]


In [ ]:
# Merge on 'asin'
df = pd.merge(us_reviews, metadata, on='asin')


In [ ]:
# Keep only products with >= 30 reviews
eligible_asins = df['asin'].value_counts()[df['asin'].value_counts() >= 30].index
df = df[df['asin'].isin(eligible_asins)]



In [ ]:
from rapidfuzz import process, fuzz
# Define brand lookup lists (expandable)
global_brands = ["Apple", "Samsung", "Sony", "Microsoft", "Nestlé"]
local_brands = ["Trader Joe's", "Kroger", "HEB", "Meijer", "Publix"]


In [ ]:
def classify_brand(brand_name):
    if pd.isna(brand_name):
        return "Unclassified"
    global_match = process.extractOne(brand_name, global_brands, scorer=fuzz.token_sort_ratio)
    local_match = process.extractOne(brand_name, local_brands, scorer=fuzz.token_sort_ratio)
    if global_match and global_match[1] >= 85:
        return "Global"
    elif local_match and local_match[1] >= 85:
        return "Local"
    else:
        return "Unclassified"
df['Brand Origin'] = df['brand'].apply(classify_brand)
df = df[df['Brand Origin'] != "Unclassified"]


In [ ]:
# Define temporal lexicons
past_keywords = [
    "heritage", "traditional", "since", "established", "legacy",
    "classic", "vintage", "timeless", "retro", "crafted",
    "roots", "decades", "century"
]

future_keywords = [
    "future", "next-gen", "innovation", "cutting-edge", "modern",
    "upcoming", "advanced", "revolutionary", "smart", "AI-powered",
    "2025", "latest", "forward"
]


In [ ]:
def detect_temporal_framing(text):
    if pd.isna(text): return "Unclassified"
    text = text.lower()
    past_hits = sum(1 for word in past_keywords if word in text)
    future_hits = sum(1 for word in future_keywords if word in text)
    if past_hits > future_hits:
        return "Past"
    elif future_hits > past_hits:
        return "Future"
    else:
        return "Unclassified"


In [ ]:
df['Temporal Frame'] = df['title'].apply(detect_temporal_framing)
df = df[df['Temporal Frame'] != "Unclassified"]
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
sid = SentimentIntensityAnalyzer()
def compute_sentiment(texts):
    scores = [sid.polarity_scores(str(text))['compound'] for text in texts]
    return np.mean(scores)

grouped = df.groupby(['asin', 'Brand Origin', 'Temporal Frame'])

results = grouped.agg({
    'overall': 'mean',
    'reviewText': compute_sentiment
}).reset_index()

results.columns = ['ASIN', 'Brand Origin', 'Temporal Frame', 'Average Star Rating', 'Average Sentiment Score']
results.to_csv("clean_symbolic_congruence_dataset.csv", index=False)
files.download("clean_symbolic_congruence_dataset.csv")


IMPORT REQUIRED LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.formula.api import ols
import statsmodels.api as sm


LOAD SIMULATED DATA

In [ ]:
df = pd.read_csv("/content/clean_symbolic_congruence_dataset.csv")

CREATE SYMBOLIC CONGRUENCE VARIABLE

In [ ]:
df["Symbolic Congruence"] = df.apply(
    lambda x: "Congruent" if (x["Brand Origin"] == "Global" and x["Temporal Frame"] == "Future") or
                               (x["Brand Origin"] == "Local" and x["Temporal Frame"] == "Past")
              else "Incongruent",
    axis=1
)



ANOVA STAR RATINGS

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

model_star = ols("Q('Average Star Rating') ~ C(Q('Brand Origin')) * C(Q('Temporal Frame'))", data=results).fit()
anova_star = sm.stats.anova_lm(model_star, typ=2)

ANOVA SENTIMENT SCORE


In [ ]:
model_sent = ols("Q('Average Sentiment Score') ~ C(Q('Brand Origin')) * C(Q('Temporal Frame'))", data=results).fit()
anova_sent = sm.stats.anova_lm(model_sent, typ=2)

ADD CONTROL VARIABLES

In [ ]:
np.random.seed(42)
df['Category'] = np.random.choice(['Electronics', 'Food', 'Cosmetics'], size=len(df))
df['Review Count'] = np.random.randint(30, 500, size=len(df))

dummies = pd.get_dummies(df['Category'], drop_first=True)
df = pd.concat([df, dummies], axis=1)

In [ ]:
np.random.seed(42)
df['Category'] = np.random.choice(['Electronics', 'Food', 'Cosmetics'], size=len(df))
df['Review Count'] = np.random.randint(30, 500, size=len(df))

dummies = pd.get_dummies(df['Category'], drop_first=True)
df = pd.concat([df, dummies], axis=1)

REGRESSION STAR RATING

In [ ]:
reg_star = ols("Q('Average Star Rating') ~ C(Q('Brand Origin')) * C(Q('Temporal Frame')) + Q('Review Count') + Food + Electronics", data=df).fit()

 REGRESSION SENTIMENT SCORE

In [ ]:
reg_sent = ols("Q('Average Sentiment Score') ~ C(Q('Brand Origin')) * C(Q('Temporal Frame')) + Q('Review Count') + Food + Electronics", data=df).fit()

PRINT RESULTS

In [ ]:
print("=== ANOVA Results: Star Ratings ===")
print(anova_star, "\n")




In [ ]:
print("=== ANOVA Results: Sentiment Score ===")
print(anova_sent, "\n")


In [ ]:
print("=== OLS Regression: Sentiment Score ===")
print(reg_sent.summary())


In [ ]:
print(results.groupby(['Brand Origin', 'Temporal Frame']).size())

In [ ]:
print(merged_df['Symbolic Congruence'].value_counts())